In [7]:
import numpy as np
import pandas as pd
from datetime import datetime

import data as Data
from indicators import SMA
import signals as Signal
from portfolio import Portfolio

data = Data.get_candles('db', range(1, 10), 1)
data

,open,high,low,close,volume,open,high,low,close,volume,...,open,high,low,close,volume,open,high,low,close,volume
,EURUSD,EURUSD,EURUSD,EURUSD,EURUSD,AUDCAD,AUDCAD,AUDCAD,AUDCAD,AUDCAD,...,CHFJPY,CHFJPY,CHFJPY,CHFJPY,CHFJPY,EURAUD,EURAUD,EURAUD,EURAUD,EURAUD
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-12-26 01:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,166.060,166.072,166.058,166.068,0.0,NaN,NaN,NaN,NaN,NaN
2023-12-26 01:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,166.068,166.074,166.025,166.032,0.0,NaN,NaN,NaN,NaN,NaN
2023-12-26 01:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,166.018,166.051,165.935,166.046,0.0,NaN,NaN,NaN,NaN,NaN
2023-12-26 01:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,166.046,166.048,166.020,166.030,0.0,NaN,NaN,NaN,NaN,NaN
2023-12-26 01:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,166.030,166.041,166.011,166.039,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-10 20:50:00,1.07691,1.07694,1.07687,1.07687,112.0,0.90277,0.90278,0.90274,0.90275,36.0,...,171.830,171.865,171.830,171.863,68.0,1.63093,1.63098,1.63086,1.63088,114.0
2024-05-10 20:51:00,1.07689,1.07689,1.07684,1.07686,64.0,0.90277,0.90280,0.90274,0.90274,39.0,...,171.866,171.870,171.858,171.863,70.0,1.63089,1.63097,1.63081,1.63097,103.0


In [9]:
sma = SMA(data['close'], windows=[10, 20]).run()
sma

SMA_10                                          \
                       EURUSD    AUDCAD    AUDCHF    AUDJPY    AUDNZD   
timestamp                                                               
2023-12-26 01:22:00       NaN       NaN       NaN       NaN       NaN   
2023-12-26 01:23:00       NaN       NaN       NaN       NaN       NaN   
2023-12-26 01:24:00       NaN       NaN       NaN       NaN       NaN   
2023-12-26 01:25:00       NaN       NaN       NaN       NaN       NaN   
2023-12-26 01:26:00       NaN       NaN       NaN       NaN       NaN   
...                       ...       ...       ...       ...       ...   
2024-05-10 20:50:00  1.076946  0.902858  0.598558  102.8666  1.097011   
2024-05-10 20:51:00  1.076926  0.902848  0.598546  102.8658  1.097017   
2024-05-10 20:52:00  1.076916  0.902835  0.598536  102.8651  1.097018   
2024-05-10 20:53:00  1.076907  0.902818  0.598525  102.8651  1.097027   
2024-05-10 20:54:00  1.076907  0.902798  0.598519  102.8651  1.097031   

                                                               SMA_20  \
                       AUDUSD    CADCHF    CHFJPY    EURAUD    EURUSD   
timestamp                                                               
2023-12-26 01:22:00       NaN       NaN       NaN       NaN       NaN   
2023-12-26 01:23:00       NaN       NaN       NaN       NaN       NaN   
2023-12-26 01:24:00       NaN       NaN       NaN       NaN       NaN   
2023-12-26 01:25:00       NaN       NaN       NaN       NaN       NaN   
2023-12-26 01:26:00       NaN       NaN       NaN       NaN       NaN   
...                       ...       ...       ...       ...       ...   
2024-05-10 20:50:00  0.660383  0.662912  171.8398  1.630734  1.077024   
2024-05-10 20:51:00  0.660363  0.662908  171.8412  1.630749  1.077008   
2024-05-10 20:52:00  0.660349  0.662906  171.8427  1.630769  1.076993   
2024-05-10 20:53:00  0.660335  0.662904  171.8445  1.630784  1.076982   
2024-05-10 20:54:00  0.660325  0.662914  171.8450  1.630803  1.076977   

                                                                        \
                       AUDCAD    AUDCHF     AUDJPY    AUDNZD    AUDUSD   
timestamp                                                                
2023-12-26 01:22:00       NaN       NaN        NaN       NaN       NaN   
2023-12-26 01:23:00       NaN       NaN        NaN       NaN       NaN   
2023-12-26 01:24:00       NaN       NaN        NaN       NaN       NaN   
2023-12-26 01:25:00       NaN       NaN        NaN       NaN       NaN   
2023-12-26 01:26:00       NaN       NaN        NaN       NaN       NaN   
...                       ...       ...        ...       ...       ...   
2024-05-10 20:50:00  0.902934  0.598564  102.87025  1.096977  0.660434   
2024-05-10 20:51:00  0.902913  0.598555  102.86935  1.096979  0.660416   
2024-05-10 20:52:00  0.902891  0.598548  102.86835  1.096977  0.660400   
2024-05-10 20:53:00  0.902868  0.598541  102.86770  1.096974  0.660385   
2024-05-10 20:54:00  0.902849  0.598539  102.86780  1.096976  0.660376   

                                                    
                       CADCHF     CHFJPY    EURAUD  
timestamp                                           
2023-12-26 01:22:00       NaN        NaN       NaN  
2023-12-26 01:23:00       NaN        NaN       NaN  
2023-12-26 01:24:00       NaN        NaN       NaN  
2023-12-26 01:25:00       NaN        NaN       NaN  
2023-12-26 01:26:00       NaN        NaN       NaN  
...                       ...        ...       ...  
2024-05-10 20:50:00  0.662868  171.84630  1.630729  
2024-05-10 20:51:00  0.662875  171.84665  1.630747  
2024-05-10 20:52:00  0.662883  171.84690  1.630760  
2024-05-10 20:53:00  0.662890  171.84730  1.630775  
2024-05-10 20:54:00  0.662902  171.84720  1.630788  

[140495 rows x 18 columns]

In [10]:
close_above_sma = Signal.above(data['close'], sma['SMA_20'])
crossover = Signal.crossover(sma['SMA_10'], sma['SMA_20'])

buy_signals = crossover & close_above_sma
buy_signals

,EURUSD,AUDCAD,AUDCHF,AUDJPY,AUDNZD,AUDUSD,CADCHF,CHFJPY,EURAUD
timestamp,,,,,,,,,
2023-12-26 01:22:00,False,False,False,False,False,False,False,False,False
2023-12-26 01:23:00,False,False,False,False,False,False,False,False,False
2023-12-26 01:24:00,False,False,False,False,False,False,False,False,False
2023-12-26 01:25:00,False,False,False,False,False,False,False,False,False
2023-12-26 01:26:00,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
2024-05-10 20:50:00,False,False,False,False,False,False,False,False,False
2024-05-10 20:51:00,False,False,False,False,False,False,False,False,False
2024-05-10 20:52:00,False,False,False,False,False,False,False,False,False


In [11]:
crossunder = Signal.crossunder(sma['SMA_10'], sma['SMA_20'])

sell_signals = crossunder
sell_signals

,EURUSD,AUDCAD,AUDCHF,AUDJPY,AUDNZD,AUDUSD,CADCHF,CHFJPY,EURAUD
timestamp,,,,,,,,,
2023-12-26 01:22:00,False,False,False,False,False,False,False,False,False
2023-12-26 01:23:00,False,False,False,False,False,False,False,False,False
2023-12-26 01:24:00,False,False,False,False,False,False,False,False,False
2023-12-26 01:25:00,False,False,False,False,False,False,False,False,False
2023-12-26 01:26:00,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
2024-05-10 20:50:00,False,False,False,False,False,False,False,False,False
2024-05-10 20:51:00,False,False,False,False,False,False,False,False,False
2024-05-10 20:52:00,False,False,False,False,False,False,False,False,False


In [12]:
pf = Portfolio.from_signals(data, buy_signals, sell_signals)
pf.get_stats()

,Profit (%),Profit ($)
EURUSD,-3.70,-37.01
AUDCAD,-9.52,-95.16
AUDCHF,-14.38,-143.85
AUDJPY,-6.82,-68.25
AUDNZD,-10.43,-104.34
AUDUSD,-5.49,-54.87
CADCHF,-6.73,-67.31
CHFJPY,-4.77,-47.65
EURAUD,-6.23,-62.32
